#### Part 4: Take the priors calculated in Part 3 and build the MCMC chains for the TVC02 sites

Benoit Montpetit, CPS/CRD/ECCC, 2025  
Julien Meloche, CPS/CRD/ECCC, 2025  
Mike Brady, CPS/CRD/ECCC, 2025  

This notebook builds the MCMC model architecture around the Snow Microwave Radiative Transfer model ([SMRT](https://github.com/smrt-model/smrt); [Picard et al., 2018](https://gmd.copernicus.org/articles/11/2763/2018/)) and runs it for the Trail Valley Creek, January campaign sites ([Montpetit et al., 2024](https://egusphere.copernicus.org/preprints/2024/egusphere-2024-651/)), using the priors built from the microwave equivalent snowpacks [(Meloche et al., Preprint)](https://egusphere.copernicus.org/preprints/2024/egusphere-2024-3169/) of SVS-2

In [ ]:
import os
os.environ['PYTENSOR_FLAGS']='blas__ldflags=-lmkl -lguide -lpthread, optimizer=fast_compile, exception_verbosity=high'
print(os.environ['PYTENSOR_FLAGS'])

import arviz as az
import numpy as np
import pymc as pm
import pytensor.tensor as pt
from pytensor.graph import Apply, Op
from pytensor import dprint
from matplotlib import pyplot as plt
import pytensor
import xarray as xr

print(f"Running on PyMC v{pm.__version__}")

%config InlineBackend.figure_format = 'retina'
az.style.use("arviz-darkgrid")

'''
Imports for running SMRT
'''
import pandas as pd
from smrt.core.globalconstants import PERMITTIVITY_OF_AIR
from smrt import sensor_list, make_model, make_snowpack, make_soil

from datetime import date

In [ ]:
%config InlineBackend.figure_format = 'retina'
az.style.use("arviz-darkgrid")

Function to read soil properties and measured backscatter for a given site

In [ ]:
def load_site_date(site):

    # UMASS KuBand data
    df_sig0 = pd.read_pickle('Data/UMass_TVC18-19_DB.pkl')
    df_sig0 = df_sig0.loc[(df_sig0.site_id==site) & (df_sig0.inc_mean<50) & (df_sig0.inc_mean>25)]
    df_sig0 = df_sig0.sample(n=4, weights=df_sig0.inc_mean)
    df_sig0 = df_sig0.loc[(df_sig0.site_id==site)]
    # df_sig0 = df_sig0.loc[(df_sig0.inc_mean-35).abs().idxmin()] #used in the case of 1 obs only
    df_sig0 = df_sig0.sample(n=4, weights=df_sig0.inc_mean)

    sig0_obs = 10*np.log10(df_sig0['slc0_sig0_filt'].values)
    # sig0_obs = 10*np.log10(df_sig0['slc0_sig0_filt']) #used in the case of 1 obs only
    
    #Optimized parameters of Montpetit et al., 2024
    df_params = pd.read_json('Data/TVC_Ku-Band_MedSnowpit.json')
    df_params = df_params.loc[df_params.site==site]
    epsrs = np.array([2.41, 3.82])
    epsr = epsrs[np.abs(epsrs-df_params.epsr_ku.values).argmin()]
    
    #Snowpit data
    df_snow = pd.read_json('Data/df_stat_pits.json')
    temperatures = df_snow[site].temperature    

    #Initialize the background part of the snowpack
    sub = make_soil('geometrical_optics', 
                    permittivity_model = complex(epsr,0.74), 
                    mean_square_slope=0.01, 
                    temperature = temperatures[-1])

    #Initialize the sensor
    sensor  = sensor_list.active(13.285e9, df_sig0.inc_mean)

    return temperatures, sig0_obs, sub, sensor

Function to simulate backscatter ($\sigma^0$) using SMRT [(Picard et al., 2018)](https://gmd.copernicus.org/articles/11/2763/2018/) and returns the log-normal difference between the measured and simulated $\sigma^0$

In [ ]:
def smrtSim(thickness_r, thickness_h, density_r, density_h, ssa_r, ssa_h, sigma, sig0_obs):

    # Creating the snowpack to simulate with the substrate
    sp = make_snowpack(thickness=np.array([thickness_r[0],thickness_h[0]]), 
                       microstructure_model='exponential',
                       density= [density_r[0], density_h[0]],
                       temperature=temperatures,
                       ice_permittivity_model=None,
                       background_permittivity_model=PERMITTIVITY_OF_AIR,
                       liquid_water=0, salinity=0, 
                       corr_length = [0.74*4*(1-density_r[0]/917)/917/ssa_r[0],1.11*4*(1-density_h[0]/917)/917/ssa_h[0]],
                       substrate = sub)

    # run SMRT
    sigma_nought = smrt_model.run(sensor, sp)    

    #Returns the log-normal difference between the observations and the simulations
    return -0.5 * ((sig0_obs - sigma_nought.sigmaVV_dB()) / sigma) ** 2 - np.log(np.sqrt(2 * np.pi)) - np.log(sigma)

Pytensor Operator used in PyMC to run MCMC optimization and use SMRT

In [ ]:
# define a pytensor Op for our likelihood function
class SMRT(Op):

    # Makes a tensor node for every MCMC iteration
    def make_node(self, thickness, h_frac, density_r, density_h,
                  ssa_r, ssa_h, sigma, data) -> Apply: 

        #converts the inputs into tensors
        thickness=pt.as_tensor(thickness)
        h_frac=pt.as_tensor(h_frac)
        density_r=pt.as_tensor(density_r)
        density_h=pt.as_tensor(density_h)
        ssa_r=pt.as_tensor(ssa_r)
        ssa_h=pt.as_tensor(ssa_h)
        data=pt.as_tensor(data)
        sigma=pt.as_tensor(sigma)

        #concatenates the inputs
        inputs = [thickness, h_frac, 
                  density_r, density_h,
                  ssa_r, ssa_h, 
                  sigma, 
                  data
                 ]

        #initiate the outputs with the same shape as the measurements input
        outputs = [data.type()]

        #returns inputs/outputs
        return Apply(self, inputs, outputs)

    # Function to perform at every MCMC iterations
    def perform(self, node: Apply, inputs: list[np.ndarray], outputs: list[list[None]]) -> None:
        
        # gets the inputs from the node
        thickness, h_frac, density_r, density_h, ssa_r, ssa_h, sigma, data = inputs

        # call the smrt function
        logl = smrtSim(thickness, h_frac, density_r, density_h, ssa_r, ssa_h, sigma, sig0_obs)

        outputs[0][0] = np.asarray(logl)

Randomizer function to allow sampling from prior distributions in the MCMC model

In [ ]:
from typing import Optional, Tuple

def random(
    h_r: np.ndarray | float,
    h_h: np.ndarray | float,
    density_r: np.ndarray | float,
    density_h: np.ndarray | float,
    ssa_r: np.ndarray | float,
    ssa_h: np.ndarray | float,
    sigma: np.ndarray | float,
    rng: Optional[np.random.Generator] = None,
    size : Optional[Tuple[int]]=None,
) -> list[np.ndarray] | float :
    return [rng.normal(loc=h_r, scale=1, size=size),
            rng.normal(loc=h_h, scale=1, size=size),
            rng.normal(loc=density_r, scale=1, size=size),
            rng.normal(loc=density_h, scale=1, size=size),
            rng.normal(loc=ssa_r, scale=1, size=size),
            rng.normal(loc=ssa_h, scale=1, size=size),
            rng.normal(loc=sigma, scale=1, size=size)]


def custom_dist_loglike(sig0_obs, h_snow, h_frac, 
                                  density_r, density_h,
                                  ssa_r, ssa_h, sigma): 
    return sig0_sim(h_snow, h_frac,density_r, density_h,
                ssa_r, ssa_h, sigma, sig0_obs)

Extracting the list of sites to process  
Note: site SC was removed since there was a corner reflector for calibration purposes in the footprint that caused artificially boosted $\sigma^0$ values [(see Montpetit et al., 2024)](https://tc.copernicus.org/articles/18/3857/2024/)

In [ ]:
from constants import TVC02

sites = pd.DataFrame({'site':TVC02})
sites.replace({'RS':'RP'}, regex=True, inplace=True)
sites=list(sites.site.values)
sites.remove('SC02')

Initiating MCMC model properties

In [ ]:
#Number of samples to select to generate a prior distribution within the MCMC model
N=1000
#Initiating Random seed to always keep the same values for all tests
RANDOM_SEED = 58
rng = np.random.default_rng(RANDOM_SEED)
#Number of iterations
num_draws=5000
#Number of chains to run in parallel
num_chains=7
#Number of cores to use in parallel processing
num_cores=56
#Number of burn-in iterations
num_tunes=1000

Initializing the SMRT model to run IBA [(Mätzler, 1998)](https://doi.org/10.1063/1.367496) and the Discrete Ordinate Radiative Transfer [(Picard et al., 2018)](https://gmd.copernicus.org/articles/11/2763/2018/) solver

In [ ]:
#Initialize the IBA/DORT SMRT model
smrt_model = make_model("iba", "dort", rtsolver_options = {'error_handling':'nan', 'phase_normalization' : True, 
                                                           'diagonalization_method':'shur_forcedtriu'})

# Runs using the ensemble means and spread of the 120 ensemble members of the Arctic version of SVS-2/Crocus

Loading the prior statistics from all 120 ensemble members of the Arctic SVS-2

In [ ]:
priors = xr.open_dataset('Data/SVS-2_ArcticPriors.nc')

In [ ]:
priors.to_dataframe()

|  |  | mean | std | min | max |
| --- | --- | --- | --- | --- | --- | 
| property | grain_type |  |  |  |  |	
| density | H |	200.288968 | 40.763197 | 100.00 | 350 |
|  | R | 246.468558 | 20.432164 | 150.00  | 450 |
| ssa | H | 4.013722 | 1.506141 | 8.00 | 25 |
|  | R | 11.571431 | 2.847040 | 10.00 | 50 |
| thickness	| H | 0.137284 | 0.082829 | 0.05 | 1 |
|  | R | 0.270772 | 0.099468 | 0.05 | 1 |


Make the MCMC model and run it

In [ ]:
for site in sites:

    #Get site specific soil and backscatter data
    temperatures, sig0_obs, sub, sensor = load_site_date(site)
    
    # outname = f'Data/MCMC_Output_{site}_{len(sig0_obs)}obs_Arctic' #used in the case of 4 obs.
    outname = f'Data/MCMC_Output_{site}_{1}obs_Arctic'

    # this list is used due to wall time issues in processing on ECCC HPC for R&D projects
    if site not in [#'RP16','RP17','RP18','RP19','RP20','RP21','RP22','RP23','RP24','RP25',
                   #'RP26','RP27','RP28','RP29','RP30','RP31','SD02','SM02','SO02','ST02','SV02',
                   ]: 

        #initate SMRT PyTensor function
        sig0_sim = SMRT()
    
        # Create the MCMC model and its priors
        with pm.Model() as mcmc_model:
        
            #Thickness prior for the rounded grain layer
            h_r = pm.TruncatedNormal("Thickness_R", mu=priors.sel(property='thickness',grain_type='R')['mean'].values, 
                                     sigma=priors.sel(property='thickness',grain_type='R')['std'].values,
                                     lower=priors.sel(property='thickness',grain_type='R')['min'].values, 
                                     upper=priors.sel(property='thickness',grain_type='R')['max'].values)
            #Thickness prior for the depth hoar layer
            h_h = pm.TruncatedNormal("Thickness_H", mu=priors.sel(property='thickness',grain_type='H')['mean'].values, 
                                     sigma=priors.sel(property='thickness',grain_type='H')['std'].values, 
                                     lower=priors.sel(property='thickness',grain_type='H')['min'].values, 
                                     upper=priors.sel(property='thickness',grain_type='H')['max'].values)
            #Constraint on the thickness between the two layers (R<=DH)
            pm.Potential('ThicknessFraction',pm.math.log(pm.math.switch(h_h>=h_r, 1, 0)))

            #Density prior for the rounded grain layer
            density_r = pm.TruncatedNormal('Density_R', mu=priors.sel(property='density',grain_type='R')['mean'].values, 
                                           sigma=priors.sel(property='density',grain_type='R')['std'].values, 
                                           lower=priors.sel(property='density',grain_type='R')['min'].values, 
                                           upper=priors.sel(property='density',grain_type='R')['max'].values)
            #Density prior for the depth hoar layer
            density_h = pm.TruncatedNormal('Density_H', mu=priors.sel(property='density',grain_type='H')['mean'].values, 
                                           sigma=priors.sel(property='density',grain_type='H')['std'].values, 
                                           lower=priors.sel(property='density',grain_type='H')['min'].values, 
                                           upper=priors.sel(property='density',grain_type='H')['max'].values)
            #Constraint on density between the two layers (R>=DH)
            pm.Potential('DensityCond',pm.math.log(pm.math.switch(density_r>=density_h, 1, 0)))

            #SSA prior for the rounded grain layer
            ssa_r = pm.TruncatedNormal('SSA_R', mu=priors.sel(property='ssa',grain_type='R')['mean'].values, 
                                       sigma=3*priors.sel(property='ssa',grain_type='R')['std'].values, 
                                       lower=priors.sel(property='ssa',grain_type='R')['min'].values, 
                                       upper=priors.sel(property='ssa',grain_type='R')['max'].values)
            #SSA prior for the depth hoar layer
            ssa_h = pm.TruncatedNormal('SSA_H', mu=priors.sel(property='ssa',grain_type='H')['mean'].values, 
                                       sigma=3*priors.sel(property='ssa',grain_type='H')['std'].values, 
                                       lower=priors.sel(property='ssa',grain_type='H')['min'].values, 
                                       upper=priors.sel(property='ssa',grain_type='H')['max'].values)
            #Constraint on the SSA between the two layers (R>=H)
            pm.Potential('SSACond',pm.math.log(pm.math.switch(ssa_r>=ssa_h, 1, 0)))

            #Measurement uncertainty prior
            sigma = pm.TruncatedNormal('Sigma', mu=1, sigma=1, lower=0.3, upper=2)

            #Likelihood function to calculate log-normal likelihood between simulated and measured backscatter
            likelihood = pm.CustomDist('likelihood', h_r, h_h, 
                                                  density_r, density_h,
                                                  ssa_r, ssa_h,
                                                  sigma,
                                                  observed=sig0_obs, logp=custom_dist_loglike, random=random)

            #Sample from prior distributions to store prior information in output file
            idata=pm.sample_prior_predictive(samples=num_tunes)

        #Run MCMC optimization
        with mcmc_model:

            #Run MCMC with DEMZ sampler with previously initiated properties
            idata.extend(pm.sample(draws=num_draws,tune=num_tunes,chains=num_chains,cores=num_chains, step=pm.step_methods.DEMetropolisZ(), blas_cores=num_cores, discard_tuned_samples=False))

        #Save MCMC prior/posterior information in NetCDF
        idata.to_netcdf(outname + '.nc')

# Runs using the top 30 ensembles means and spread of the Arctic version of SVS-2/Crocus [Woolley et al., 2024](https://tc.copernicus.org/articles/18/5685/2024/tc-18-5685-2024-discussion.html)

In [ ]:
priors = xr.open_dataset('Data/SVS-2_ArcticPriorsTop30.nc')
priors.to_dataframe()

|  |  | mean | std | min | max |
| --- | --- | --- | --- | --- | ---| 
| property | grain_type |  |  |  |  |
| density | H | 199.040639 | 35.983911 | 100.00 | 350 |
|  | R | 235.424924 | 8.233131 | 150.00 | 450 |
| ssa | H | 3.982570 | 1.454134 | 8.00 | 25 |
|  | R | 11.187591 | 2.466457 | 10.00 | 50 |
| thickness | H | 0.132921 | 0.078534 | 0.05 | 1 |
|  | R | 0.285382 | 0.094618 | 0.05 | 1 |


In [ ]:
for site in sites:

    print(site)

    temperatures, sig0_obs, sub, sensor = load_site_date(site)
    
    outname = f'Data/MCMC_Output_{site}_{1}obs_ArcticTop30'
    
    if site not in [#'RP16','RP17','RP18','RP19','RP20','RP21','RP22','RP23','RP24','RP25',
                   #'RP26','RP27','RP28','RP29','RP30','RP31','SD02','SM02','SO02','ST02','SV02',
                   ]:
    
        sig0_sim = SMRT()
    
        # use PyMC to sample from log-likelihood
        with pm.Model() as mcmc_model:
        
            #Woolley et al., 2024
            h_r = pm.TruncatedNormal("Thickness_R", mu=priors.sel(property='thickness',grain_type='R')['mean'].values, 
                                     sigma=priors.sel(property='thickness',grain_type='R')['std'].values,
                                     lower=priors.sel(property='thickness',grain_type='R')['min'].values, 
                                     upper=priors.sel(property='thickness',grain_type='R')['max'].values)
            h_h = pm.TruncatedNormal("Thickness_H", mu=priors.sel(property='thickness',grain_type='H')['mean'].values, 
                                     sigma=priors.sel(property='thickness',grain_type='H')['std'].values, 
                                     lower=priors.sel(property='thickness',grain_type='H')['min'].values, 
                                     upper=priors.sel(property='thickness',grain_type='H')['max'].values)
            pm.Potential('ThicknessFraction',pm.math.log(pm.math.switch(h_h>=h_r, 1, 0)))
            density_r = pm.TruncatedNormal('Density_R', mu=priors.sel(property='density',grain_type='R')['mean'].values, 
                                           sigma=priors.sel(property='density',grain_type='R')['std'].values, 
                                           lower=priors.sel(property='density',grain_type='R')['min'].values, 
                                           upper=priors.sel(property='density',grain_type='R')['max'].values)
            density_h = pm.TruncatedNormal('Density_H', mu=priors.sel(property='density',grain_type='H')['mean'].values, 
                                           sigma=priors.sel(property='density',grain_type='H')['std'].values, 
                                           lower=priors.sel(property='density',grain_type='H')['min'].values, 
                                           upper=priors.sel(property='density',grain_type='H')['max'].values)
            pm.Potential('DensityCond',pm.math.log(pm.math.switch(density_r>=density_h, 1, 0)))
            ssa_r = pm.TruncatedNormal('SSA_R', mu=priors.sel(property='ssa',grain_type='R')['mean'].values, 
                                       sigma=3*priors.sel(property='ssa',grain_type='R')['std'].values, 
                                       lower=priors.sel(property='ssa',grain_type='R')['min'].values, 
                                       upper=priors.sel(property='ssa',grain_type='R')['max'].values)
            ssa_h = pm.TruncatedNormal('SSA_H', mu=priors.sel(property='ssa',grain_type='H')['mean'].values, 
                                       sigma=3*priors.sel(property='ssa',grain_type='H')['std'].values, 
                                       lower=priors.sel(property='ssa',grain_type='H')['min'].values, 
                                       upper=priors.sel(property='ssa',grain_type='H')['max'].values)
            pm.Potential('SSACond',pm.math.log(pm.math.switch(ssa_r>=ssa_h, 1, 0)))
            sigma = pm.TruncatedNormal('Sigma', mu=1, sigma=1, lower=0.3, upper=2)
        
            likelihood = pm.CustomDist('likelihood', h_r, h_h, 
                                                  density_r, density_h,
                                                  ssa_r, ssa_h,
                                                  sigma,
                                                  observed=sig0_obs, logp=custom_dist_loglike, random=random)
            idata=pm.sample_prior_predictive(samples=num_tunes)
        
        with mcmc_model:
        
            idata.extend(pm.sample(draws=num_draws,tune=num_tunes,chains=num_chains,cores=num_chains, step=pm.step_methods.DEMetropolisZ(), blas_cores=num_cores, discard_tuned_samples=False))
        
        idata.to_netcdf(outname + '.nc')

# These runs are for the unconstrained Arctic Test

In [ ]:
for site in sites:

    print(site)

    temperatures, sig0_obs, sub, sensor = load_site_date(site)
    
    outname = f'Data/MCMC_Outputs/MCMC_Output_{site}_{len(sig0_obs)}obs_Arctic_unconstrained'

    if site not in [#'RP16','RP17','RP18','RP19','RP20','RP21','RP22','RP23','RP24','RP25',
                    #'RP26','RP27','RP28','RP29','RP30','RP31','SD02','SM02','SO02','ST02','SV02',
                   ]:
    
        sig0_sim = SMRT()
    
        # use PyMC to sample from log-likelihood
        with pm.Model() as mcmc_model:
        
            #Woolley et al., 2024
            h_r = pm.TruncatedNormal("Thickness_R", mu=priors.sel(property='thickness',grain_type='R')['mean'].values, 
                                     sigma=2*priors.sel(property='thickness',grain_type='R')['std'].values,
                                     lower=priors.sel(property='thickness',grain_type='R')['min'].values, 
                                     upper=priors.sel(property='thickness',grain_type='R')['max'].values)
            h_h = pm.TruncatedNormal("Thickness_H", mu=priors.sel(property='thickness',grain_type='H')['mean'].values, 
                                     sigma=2*priors.sel(property='thickness',grain_type='H')['std'].values, 
                                     lower=priors.sel(property='thickness',grain_type='H')['min'].values, 
                                     upper=priors.sel(property='thickness',grain_type='H')['max'].values)
            density_r = pm.TruncatedNormal('Density_R', mu=priors.sel(property='density',grain_type='R')['mean'].values, 
                                           sigma=1.5*103,#priors.sel(property='density',grain_type='R')['std'].values, 
                                           lower=priors.sel(property='density',grain_type='R')['min'].values, 
                                           upper=priors.sel(property='density',grain_type='R')['max'].values)
            density_h = pm.TruncatedNormal('Density_H', mu=priors.sel(property='density',grain_type='H')['mean'].values, 
                                           sigma=priors.sel(property='density',grain_type='H')['std'].values, 
                                           lower=priors.sel(property='density',grain_type='H')['min'].values, 
                                           upper=priors.sel(property='density',grain_type='H')['max'].values)
            ssa_r = pm.TruncatedNormal('SSA_R', mu=priors.sel(property='ssa',grain_type='R')['mean'].values, 
                                       sigma=3*priors.sel(property='ssa',grain_type='R')['std'].values, 
                                       lower=priors.sel(property='ssa',grain_type='R')['min'].values, 
                                       upper=priors.sel(property='ssa',grain_type='R')['max'].values)
            ssa_h = pm.TruncatedNormal('SSA_H', mu=priors.sel(property='ssa',grain_type='H')['mean'].values, 
                                       sigma=3*priors.sel(property='ssa',grain_type='H')['std'].values, 
                                       lower=priors.sel(property='ssa',grain_type='H')['min'].values, 
                                       upper=priors.sel(property='ssa',grain_type='H')['max'].values)
            sigma = pm.TruncatedNormal('Sigma', mu=1, sigma=1, lower=0.3, upper=2)
        
            likelihood = pm.CustomDist('likelihood', h_r, h_h, 
                                                  density_r, density_h,
                                                  ssa_r, ssa_h,
                                                  sigma,
                                                  observed=sig0_obs, logp=custom_dist_loglike, random=random)
            idata=pm.sample_prior_predictive(samples=num_tunes)
        
        with mcmc_model:
        
            idata.extend(pm.sample(draws=num_draws,tune=num_tunes,chains=num_chains,cores=num_chains, step=pm.step_methods.DEMetropolisZ(), blas_cores=num_cores, discard_tuned_samples=False))
        
        idata.to_netcdf(outname + '.nc')

# Runs using the ensemble means and spread of the 120 ensemble members of the Default version of SVS-2/Crocus

In [ ]:
priors = xr.open_dataset('Data/SVS-2_DefaultPriors.nc')

In [ ]:
priors.to_dataframe()

|  |  | mean | std | min | max |
| --- | --- | --- | --- | --- | --- |
| property | grain_type |  |  |  |  |				
| density | H | 189.961906 | 29.981146 | 100.00 | 350 |
|  | R | 217.699286 | 14.564853 | 150.00 | 450 |
| ssa | H | 5.236867 | 1.349880 | 8.00 | 25 |
|  | R | 12.681343 | 1.600555 | 10.00 | 50 |
| thickness | H | 0.153560 | 0.040753 | 0.05 |	1 |
|  | R | 0.298376 | 0.077330 | 0.05 | 1 |

In [ ]:
for site in sites:

    print(site)

    temperatures, sig0_obs, sub, sensor = load_site_date(site)
    
    outname = f'Data/MCMC_Output_{site}_{1}obs_Default'

    if site not in [#'RP16','RP17','RP18','RP19','RP20','RP21','RP22','RP23','RP24','RP25',
                    #'RP26','RP27','RP28','RP29',#'RP30','RP31','SD02',#'SM02','SO02','ST02','SV02',
                   ]:
    
        sig0_sim = SMRT()
    
        # use PyMC to sample from log-likelihood
        with pm.Model() as mcmc_model:
        
            #Woolley et al., 2024
            h_r = pm.TruncatedNormal("Thickness_R", mu=priors.sel(property='thickness',grain_type='R')['mean'].values, 
                                     sigma=priors.sel(property='thickness',grain_type='R')['std'].values,
                                     lower=priors.sel(property='thickness',grain_type='R')['min'].values, 
                                     upper=priors.sel(property='thickness',grain_type='R')['max'].values)
            h_h = pm.TruncatedNormal("Thickness_H", mu=priors.sel(property='thickness',grain_type='H')['mean'].values, 
                                     sigma=priors.sel(property='thickness',grain_type='H')['std'].values, 
                                     lower=priors.sel(property='thickness',grain_type='H')['min'].values, 
                                     upper=priors.sel(property='thickness',grain_type='H')['max'].values)
            pm.Potential('ThicknessFraction',pm.math.log(pm.math.switch(h_h>=h_r, 1, 0)))
            density_r = pm.TruncatedNormal('Density_R', mu=priors.sel(property='density',grain_type='R')['mean'].values, 
                                           sigma=priors.sel(property='density',grain_type='R')['std'].values, 
                                           lower=priors.sel(property='density',grain_type='R')['min'].values, 
                                           upper=priors.sel(property='density',grain_type='R')['max'].values)
            density_h = pm.TruncatedNormal('Density_H', mu=priors.sel(property='density',grain_type='H')['mean'].values, 
                                           sigma=priors.sel(property='density',grain_type='H')['std'].values, 
                                           lower=priors.sel(property='density',grain_type='H')['min'].values, 
                                           upper=priors.sel(property='density',grain_type='H')['max'].values)
            pm.Potential('DensityCond',pm.math.log(pm.math.switch(density_r>=density_h, 1, 0)))
            ssa_r = pm.TruncatedNormal('SSA_R', mu=priors.sel(property='ssa',grain_type='R')['mean'].values, 
                                       sigma=3*priors.sel(property='ssa',grain_type='R')['std'].values, 
                                       lower=priors.sel(property='ssa',grain_type='R')['min'].values, 
                                       upper=priors.sel(property='ssa',grain_type='R')['max'].values)
            ssa_h = pm.TruncatedNormal('SSA_H', mu=priors.sel(property='ssa',grain_type='H')['mean'].values, 
                                       sigma=3*priors.sel(property='ssa',grain_type='H')['std'].values, 
                                       lower=priors.sel(property='ssa',grain_type='H')['min'].values, 
                                       upper=priors.sel(property='ssa',grain_type='H')['max'].values)
            pm.Potential('SSACond',pm.math.log(pm.math.switch(ssa_r>=ssa_h, 1, 0)))
            sigma = pm.TruncatedNormal('Sigma', mu=1, sigma=1, lower=0.3, upper=2)
        
            likelihood = pm.CustomDist('likelihood', h_r, h_h, 
                                                  density_r, density_h,
                                                  ssa_r, ssa_h,
                                                  sigma,
                                                  observed=sig0_obs, logp=custom_dist_loglike, random=random)
            idata=pm.sample_prior_predictive(samples=num_tunes)
        
        with mcmc_model:
        
            idata.extend(pm.sample(draws=40000,tune=num_tunes,chains=num_chains,cores=num_chains, step=pm.step_methods.DEMetropolisZ(), blas_cores=num_cores, discard_tuned_samples=False))
            
        idata.to_netcdf(outname + '.nc')

# Runs using the top 30 ensembles means and spread of the Default version of SVS-2/Crocus [(see Woolley et al.,2024)](https://tc.copernicus.org/articles/18/5685/2024/tc-18-5685-2024.html)

In [ ]:
priors = xr.open_dataset('Data/SVS-2_DefaultPriorsTop30.nc')

In [ ]:
priors.to_dataframe()

|  |  | mean | std | min | max |
| --- | --- | --- | --- | --- | --- |
| property | grain_type |  |  |  |  |
| density | H | 218.688528 | 3.185332 | 100.00 | 350 |
|  | R | 231.133920 | 5.215652 | 150.00 | 450 |
| ssa | H | 6.109727 | 0.761701 | 8.00 | 25 |
|  | R | 12.097775 | 1.404093 | 10.00 | 50 |
| thickness | H | 0.158499 | 0.028717 | 0.05 | 1 |
|  | R |0.242738 | 0.027913 | 0.05 |1 |


In [ ]:
for site in sites:

    print(site)

    temperatures, sig0_obs, sub, sensor = load_site_date(site)

    outname = f'Data/MCMC_Output_{site}_{1}obs_DefaultTop30'

    if site not in [#'RP16','RP17','RP18','RP19','RP20','RP21','RP22','RP23','RP24','RP25',
                    #'RP26','RP27','RP28','RP29','RP30','RP31','SD02','SM02','SO02','ST02','SV02',
                   ]:
    
        sig0_sim = SMRT()
    
        # use PyMC to sample from log-likelihood
        with pm.Model() as mcmc_model:
        
            #Woolley et al., 2024
            h_r = pm.TruncatedNormal("Thickness_R", mu=priors.sel(property='thickness',grain_type='R')['mean'].values, 
                                     sigma=priors.sel(property='thickness',grain_type='R')['std'].values,
                                     lower=priors.sel(property='thickness',grain_type='R')['min'].values, 
                                     upper=priors.sel(property='thickness',grain_type='R')['max'].values)
            h_h = pm.TruncatedNormal("Thickness_H", mu=priors.sel(property='thickness',grain_type='H')['mean'].values, 
                                     sigma=priors.sel(property='thickness',grain_type='H')['std'].values, 
                                     lower=priors.sel(property='thickness',grain_type='H')['min'].values, 
                                     upper=priors.sel(property='thickness',grain_type='H')['max'].values)
            pm.Potential('ThicknessFraction',pm.math.log(pm.math.switch(h_h>=h_r, 1, 0)))
            density_r = pm.TruncatedNormal('Density_R', mu=priors.sel(property='density',grain_type='R')['mean'].values, 
                                           sigma=priors.sel(property='density',grain_type='R')['std'].values, 
                                           lower=priors.sel(property='density',grain_type='R')['min'].values, 
                                           upper=priors.sel(property='density',grain_type='R')['max'].values)
            density_h = pm.TruncatedNormal('Density_H', mu=priors.sel(property='density',grain_type='H')['mean'].values, 
                                           sigma=priors.sel(property='density',grain_type='H')['std'].values, 
                                           lower=priors.sel(property='density',grain_type='H')['min'].values, 
                                           upper=priors.sel(property='density',grain_type='H')['max'].values)
            pm.Potential('DensityCond',pm.math.log(pm.math.switch(density_r>=density_h, 1, 0)))
            ssa_r = pm.TruncatedNormal('SSA_R', mu=priors.sel(property='ssa',grain_type='R')['mean'].values, 
                                       sigma=3*priors.sel(property='ssa',grain_type='R')['std'].values, 
                                       lower=priors.sel(property='ssa',grain_type='R')['min'].values, 
                                       upper=priors.sel(property='ssa',grain_type='R')['max'].values)
            ssa_h = pm.TruncatedNormal('SSA_H', mu=priors.sel(property='ssa',grain_type='H')['mean'].values, 
                                       sigma=3*priors.sel(property='ssa',grain_type='H')['std'].values, 
                                       lower=priors.sel(property='ssa',grain_type='H')['min'].values, 
                                       upper=priors.sel(property='ssa',grain_type='H')['max'].values)
            pm.Potential('SSACond',pm.math.log(pm.math.switch(ssa_r>=ssa_h, 1, 0)))
            sigma = pm.TruncatedNormal('Sigma', mu=1, sigma=1, lower=0.3, upper=2)
        
            likelihood = pm.CustomDist('likelihood', h_r, h_h, 
                                                  density_r, density_h,
                                                  ssa_r, ssa_h,
                                                  sigma,
                                                  observed=sig0_obs, logp=custom_dist_loglike, random=random)
            idata=pm.sample_prior_predictive(samples=num_tunes)
        
        with mcmc_model:
        
            idata.extend(pm.sample(draws=num_draws,tune=num_tunes,chains=num_chains,cores=num_chains, step=pm.step_methods.DEMetropolisZ(), blas_cores=num_cores, discard_tuned_samples=False))
            
        idata.to_netcdf(outname + '.nc')